### 保存变量
    一个简单的例子来保存weights和bias Tensors

In [1]:
import tensorflow as tf

#The file path to save the data
#文件保存路径
save_file = './model.ckpt'

#Two Tensor Variable: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))#正态分布产生随机数
bias = tf.Variable(tf.truncated_normal([3]))

#用来存取Tensor变量的类
saver = tf.train.Saver()

with tf.Session() as sess:
    #初始化所有变量
    sess.run(tf.global_variables_initializer())
    
    #显示变量和权重
    print("weights:")
    print(sess.run(weights))
    print("bias:")
    print(sess.run(bias))
    
    #保存模型
    saver.save(sess, save_file)

weights:
[[-0.23113304  0.49243954 -0.69021088]
 [ 1.29466176  1.54267168  1.31112385]]
bias:
[ 1.38376117 -0.13668728 -0.40629727]


weights 和 bias Tensor用tf.truncated_normal()函数设定了随机值。用tf.train.Saver.save()函数把这些值保存在save_file位置，
命名为"model.ckpt"

### 加载变量

In [2]:
#移除之前的权重和偏置项
tf.reset_default_graph()

#两个变量： 权重和偏置项
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()

with tf.Session() as sess:
    #加载权重和偏置项
    saver.restore(sess, save_file)
    
    #显示权重和偏置项
    print("weights:")
    print(sess.run(weights))
    print("bias:")
    print(sess.run(bias))

INFO:tensorflow:Restoring parameters from ./model.ckpt
weights:
[[-0.23113304  0.49243954 -0.69021088]
 [ 1.29466176  1.54267168  1.31112385]]
bias:
[ 1.38376117 -0.13668728 -0.40629727]


### Saved a Trained Model
保存一个训练好的模型

In [3]:
#remove previous Tensors and Operations
tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_datat_data
import numpy as np

In [4]:
#参数
learning_rate = 0.001
n_input = 784
n_classes = 10

#加载数据
mnist = input_data.read_data_sets('.', one_hot=True)

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [6]:
#特征和标签
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

#权重和偏差项
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

#logits
logits = tf.add(tf.matmul(features, weights), bias)

#损失函数和优化器
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#计算准确率
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


### 训练模型并保存权重

In [8]:
import math

save_file = './train_model.ckpt'
batch_size = 128
n_epochs = 100

saver = tf.train.Saver()

#Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            sess.run(
                optimizer,
                feed_dict = {features: batch_features, labels: batch_labels})
            
            #print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(
                accuracy,
                feed_dict = {
                    features: mnist.validation.images,
                    labels: mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                    epoch,
                    valid_accuracy))

    #save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

Epoch 0   - Validation Accuracy: 0.11219999939203262
Epoch 10  - Validation Accuracy: 0.3041999936103821
Epoch 20  - Validation Accuracy: 0.44279998540878296
Epoch 30  - Validation Accuracy: 0.5371999740600586
Epoch 40  - Validation Accuracy: 0.599399983882904
Epoch 50  - Validation Accuracy: 0.6424000263214111
Epoch 60  - Validation Accuracy: 0.6678000092506409
Epoch 70  - Validation Accuracy: 0.6904000043869019
Epoch 80  - Validation Accuracy: 0.7107999920845032
Epoch 90  - Validation Accuracy: 0.727400004863739
Trained Model Saved.


### 加载训练好的模型
验证测试集的准确率

In [9]:
saver = tf.train.Saver()

#Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)
    
    test_accuracy = sess.run(
        accuracy,
        feed_dict = {
            features: mnist.test.images,
            labels: mnist.test.labels})
print('Test Accuracy: {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./train_model.ckpt
Test Accuracy: 0.7343000173568726
